In [54]:
import torch
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn


In [55]:
#device configuration
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [56]:
#Hyperparameters
num_epochs=4
batch_size=4
learning_rate=0.001

In [57]:
#dataset has PILImage images of range [0,1].
#We transform them to Tensors of normalized range[-1,1]
data_transformer = transforms.Compose([
        # transforms.RandomResizedCrop((150,150)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
        ])

In [58]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=data_transformer)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=data_transformer)

Files already downloaded and verified
Files already downloaded and verified


In [59]:
train_loader =  torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader =  torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)
classes=('plane','car','bird','cat','deer','dog','frog','hourse','ship','truck')

In [69]:
#implement the convolution 
#never forget to call super
class ConVNet(nn.Module):
    def __init__(self):
        super(ConVNet,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
     
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x= x.view(-1,16 *5* 5)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

       
        


In [70]:
model = ConVNet().to(device)
print(model)
criteria=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

ConVNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [71]:
n_total_steps= len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
      
        images=images.to(device)
        labels=labels.to(device)

        #Forward pass
        outputs= model(images)
        loss = criteria(outputs,labels)

        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print('Finish Training')




       


      


Finish Training


In [73]:
with torch.no_grad():
    n_correct = 0
    n_sample = 0
    n_class_correct =[0 for i in range(10)]
    n_class_sample =[0 for i in range(10)]
    print(n_class_correct)
    print(n_class_sample)
    count=1
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        #max return (value,index)
        _,predicted=torch.max(outputs,1)
        n_sample +=labels.size(0)
        n_correct += (labels==predicted).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label]+=1
            n_class_sample[label]+=1


acc= 100.0 * n_correct / n_sample
print(f'Accuracy of the network:{acc} %')

for i in range(10):
    acc=100.0 * n_class_correct[i] / n_class_sample[i]
    print(f'Accuracy of the {classes[i]}:{acc} %')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy of the network:39.08 %
Accuracy of the plane:62.5 %
Accuracy of the car:67.6470588235294 %
Accuracy of the bird:13.793103448275861 %
Accuracy of the cat:27.586206896551722 %
Accuracy of the deer:9.090909090909092 %
Accuracy of the dog:36.231884057971016 %
Accuracy of the frog:33.333333333333336 %
Accuracy of the hourse:48.4375 %
Accuracy of the ship:27.77777777777778 %
Accuracy of the truck:36.8421052631579 %
